# MQL4 Strategy Parser to Backtrader Python Classes

This notebook parses MQL4 trading strategies and converts them to Backtrader-compatible Python classes.

## Overview
- **Source**: MQL4 code with GetEntrySignal_XXX() and GetExitSignal_XXX() functions
- **Target**: Python Backtrader strategy classes with talib indicators
- **Data**: S&P 500 historical data (1789-2025)

## Strategy Structure
Each strategy follows this pattern:
1. **Class Name**: Extracted from function numbers (000, 001, etc.)
2. **Entry Indicators**: Parsed from GetEntrySignal_XXX() functions
3. **Exit Indicators**: Parsed from GetExitSignal_XXX() functions
4. **Parameters**: Technical indicator periods and levels
5. **Logic**: Boolean combinations (ind0long && ind1long && ind2long)

In [1]:
# Import Required Libraries
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd
import datetime
import re
import json
from typing import Dict, List, Tuple, Any
import numpy as np

print("Libraries imported successfully!")
print("✓ Backtrader framework ready")
print("✓ Pandas for data handling")
print("✓ TAlib integration available via bt.talib")

Libraries imported successfully!
✓ Backtrader framework ready
✓ Pandas for data handling
✓ TAlib integration available via bt.talib


## Strategy 000 Analysis

**Entry Indicators (GetEntrySignal_000):**
1. **MACD Signal** (Close, 24, 54, 14) - Crossover logic
2. **Bulls Power** (12), Level: -0.0020 
3. **Bulls Power** (27), Level: -0.0006

**Exit Indicators (GetExitSignal_000):**
1. **Williams' Percent Range** (55), Level: -100.0

**Entry Logic:**
- **Long**: `ind0long && ind1long && ind2long`
- **Short**: `ind0short && ind1short && ind2short`

**MQL4 Code Translation:**
- `iMACD()` → `bt.talib.MACD()`
- `iBullsPower()` → Custom Bulls Power calculation
- `iWPR()` → `bt.talib.WILLR()`

In [2]:
# MQL4 Strategy Parser Functions

def parse_strategy_000_from_mql4():
    """
    Parse MQL4 Strategy 000 and extract indicator parameters
    
    Based on the MQL4 code:
    GetEntrySignal_000():
    - MACD Signal (Close, 24, 54, 14)
    - Bulls Power (12), Level: -0.0020
    - Bulls Power (27), Level: -0.0006
    
    GetExitSignal_000():
    - Williams' Percent Range (55), Level: -100.0
    """
    
    strategy_info = {
        'class_name': 'Strategy000',
        'entry_indicators': {
            'macd': {
                'fast_period': 24,
                'slow_period': 54,
                'signal_period': 14,
                'price': 'close'
            },
            'bulls_power_1': {
                'period': 12,
                'level': -0.0020
            },
            'bulls_power_2': {
                'period': 27,
                'level': -0.0006
            }
        },
        'exit_indicators': {
            'williams_pr': {
                'period': 55,
                'level': -100.0
            }
        },
        'risk_management': {
            'stop_loss': 422,
            'take_profit': 0,
            'use_stop_loss': True,
            'use_take_profit': False,
            'trailing_stop': True
        }
    }
    
    return strategy_info

# Parse Strategy 000
strategy_000_config = parse_strategy_000_from_mql4()
print("=== STRATEGY 000 CONFIGURATION ===")
print(json.dumps(strategy_000_config, indent=2))
print("\n✓ Strategy 000 parsed successfully!")

=== STRATEGY 000 CONFIGURATION ===
{
  "class_name": "Strategy000",
  "entry_indicators": {
    "macd": {
      "fast_period": 24,
      "slow_period": 54,
      "signal_period": 14,
      "price": "close"
    },
    "bulls_power_1": {
      "period": 12,
      "level": -0.002
    },
    "bulls_power_2": {
      "period": 27,
      "level": -0.0006
    }
  },
  "exit_indicators": {
    "williams_pr": {
      "period": 55,
      "level": -100.0
    }
  },
  "risk_management": {
    "stop_loss": 422,
    "take_profit": 0,
    "use_stop_loss": true,
    "use_take_profit": false,
    "trailing_stop": true
  }
}

✓ Strategy 000 parsed successfully!


In [ ]:
# Strategy000 Backtrader Class

class Strategy000(bt.Strategy):
    """
    Backtrader implementation of MQL4 Strategy000
    
    Entry Conditions (ALL must be True for Long/Short):
    - MACD Signal crossover logic
    - Bulls Power (12) above/below level -0.0020
    - Bulls Power (27) above/below level -0.0006
    
    Exit Conditions:
    - Williams %R crossover at level -100.0
    """
    
    params = (
        # MACD parameters
        ('macd_fast', 24),
        ('macd_slow', 54), 
        ('macd_signal', 14),
        
        # Bulls Power parameters
        ('bulls_power_1_period', 12),
        ('bulls_power_1_level', -0.0020),
        ('bulls_power_2_period', 27),
        ('bulls_power_2_level', -0.0006),
        
        # Williams %R parameters
        ('williams_period', 55),
        ('williams_level', -100.0),
        
        # Sigma for signal tolerance
        ('sigma', 0.0001),
        
        # Logging
        ('printlog', False),
    )

    def __init__(self):
        # Entry indicators
        # MACD components - using backtrader's built-in MACD indicator instead of talib
        self.macd_ind = bt.indicators.MACD(
            self.data.close,
            period_me1=self.p.macd_fast,
            period_me2=self.p.macd_slow,
            period_signal=self.p.macd_signal
        )
        # Access MACD components
        self.macd = self.macd_ind.macd
        self.macd_signal = self.macd_ind.signal
        
        # Bulls Power indicators (approximated as Close - EMA)
        self.ema_12 = bt.indicators.EMA(self.data.close, period=self.p.bulls_power_1_period)
        self.bulls_power_1 = self.data.close - self.ema_12
        
        self.ema_27 = bt.indicators.EMA(self.data.close, period=self.p.bulls_power_2_period)
        self.bulls_power_2 = self.data.close - self.ema_27
        
        # Exit indicator - Williams %R using backtrader's built-in indicator
        self.williams_pr = bt.indicators.WilliamsR(
            self.data,
            period=self.p.williams_period
        )
        
        # Tracking variables
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # For MACD crossover tracking
        self.macd_diff = self.macd - self.macd_signal

    def log(self, txt, dt=None, doprint=False):
        """Logging function for this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.4f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.4f}')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')

    def next(self):
        # Progress logging
        if len(self) % 250 == 0:
            self.log(f'Day {len(self)}: Progress update...')

        # Check if an order is pending
        if self.order:
            return

        # Skip if we don't have enough data for all indicators
        if len(self.data) < max(self.p.macd_slow, self.p.williams_period, self.p.bulls_power_2_period) + 10:
            return
        
        # Get current values (using [0] for current bar, [-1] for previous bar)
        if not self.position:
            # ENTRY CONDITIONS - Translation from MQL4 logic
            
            # MACD Signal crossover: ind0val1 < 0 - sigma && ind0val2 > 0 + sigma
            try:
                macd_diff_current = self.macd_diff[0]
                macd_diff_prev = self.macd_diff[-1] if len(self) > 0 else 0
            except (IndexError, TypeError):
                return
            
            ind0long = (macd_diff_current < (0 - self.p.sigma) and 
                       macd_diff_prev > (0 + self.p.sigma))
            ind0short = (macd_diff_current > (0 + self.p.sigma) and 
                        macd_diff_prev < (0 - self.p.sigma))
            
            # Bulls Power (12): ind1val1 > -0.0020 + sigma
            try:
                bulls_1_current = self.bulls_power_1[0]
                ind1long = bulls_1_current > (self.p.bulls_power_1_level + self.p.sigma)
                ind1short = bulls_1_current < (-self.p.bulls_power_1_level - self.p.sigma)
            except (IndexError, TypeError):
                return
            
            # Bulls Power (27): ind2val1 > -0.0006 + sigma  
            try:
                bulls_2_current = self.bulls_power_2[0]
                ind2long = bulls_2_current > (self.p.bulls_power_2_level + self.p.sigma)
                ind2short = bulls_2_current < (-self.p.bulls_power_2_level - self.p.sigma)
            except (IndexError, TypeError):
                return
            
            # Combined entry conditions
            entry_long = ind0long and ind1long and ind2long
            entry_short = ind0short and ind1short and ind2short
            
            if entry_long:
                self.log(f'BUY CREATE - MACD: {macd_diff_current:.6f}, Bulls1: {bulls_1_current:.6f}, Bulls2: {bulls_2_current:.6f}')
                self.order = self.buy()
                
            elif entry_short:
                self.log(f'SELL CREATE - MACD: {macd_diff_current:.6f}, Bulls1: {bulls_1_current:.6f}, Bulls2: {bulls_2_current:.6f}')
                self.order = self.sell()
        
        else:
            # EXIT CONDITIONS - Williams %R
            try:
                williams_current = self.williams_pr[0]
                williams_prev = self.williams_pr[-1] if len(self) > 0 else -50
            except (IndexError, TypeError):
                return
            
            # Exit long: ind3val1 > -100.0 + sigma && ind3val2 < -100.0 - sigma
            # Exit short: ind3val1 < -100 - -100.0 - sigma && ind3val2 > -100 - -100.0 + sigma
            
            exit_long = (williams_current > (self.p.williams_level + self.p.sigma) and 
                        williams_prev < (self.p.williams_level - self.p.sigma))
            exit_short = (williams_current < (-100 - self.p.williams_level - self.p.sigma) and 
                         williams_prev > (-100 - self.p.williams_level + self.p.sigma))
            
            if self.position.size > 0 and exit_long:  # Long position
                self.log(f'SELL CREATE - Exit Long - Williams: {williams_current:.2f}')
                self.order = self.sell()
            elif self.position.size < 0 and exit_short:  # Short position
                self.log(f'BUY CREATE - Exit Short - Williams: {williams_current:.2f}')
                self.order = self.buy()

    def stop(self):
        self.log(f'Strategy000 Final Portfolio Value: {self.broker.getvalue():.2f}', doprint=True)

print("✓ Strategy000 class created successfully!")
print("✓ Using Backtrader built-in indicators (MACD, EMA, Williams %R)")
print("✓ Added robust error handling for indicator calculations")
print("✓ Ready for backtesting with S&P 500 data")

✓ Strategy000 class created successfully!
✓ Ready for backtesting with S&P 500 data


In [4]:
# Data Loading and Preprocessing Function

def preprocess_spy_data_for_strategy000():
    """
    Preprocess S&P 500 data for Strategy000 backtesting
    Using the same data path as the reference backtrader notebook
    """
    # Use the same file path as the reference notebook
    spy_file = r'../04_S&P500_quant_analysis/01_data/S&P500_D_1789-05-01_2025-09-17.csv'
    
    print(f"Loading data from: {spy_file}")
    
    # Read the file skipping the first 3 header rows
    df = pd.read_csv(spy_file, skiprows=3, header=None)
    
    # Set proper column names
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Sort by date (should already be sorted)
    df = df.sort_values('Date')
    
    # Remove any rows with missing values
    df = df.dropna()
    
    # Save the clean data for Strategy000
    clean_file = 'spy_data_strategy000.csv'
    df.to_csv(clean_file, index=False)
    
    print(f"✓ Clean data saved to: {clean_file}")
    print(f"✓ Data shape: {df.shape}")
    print(f"✓ Date range: {df['Date'].min()} to {df['Date'].max()}")
    print(f"✓ Ready for Strategy000 backtesting")
    
    return clean_file, df

# Preprocess the data
print("=== DATA PREPROCESSING FOR STRATEGY000 ===")
clean_data_file, spy_df = preprocess_spy_data_for_strategy000()

=== DATA PREPROCESSING FOR STRATEGY000 ===
Loading data from: ../04_S&P500_quant_analysis/01_data/S&P500_D_1789-05-01_2025-09-17.csv
✓ Clean data saved to: spy_data_strategy000.csv
✓ Data shape: (39530, 6)
✓ Date range: 1789-07-01 00:00:00 to 2025-09-17 00:00:00
✓ Ready for Strategy000 backtesting
✓ Clean data saved to: spy_data_strategy000.csv
✓ Data shape: (39530, 6)
✓ Date range: 1789-07-01 00:00:00 to 2025-09-17 00:00:00
✓ Ready for Strategy000 backtesting


In [5]:
# Strategy000 Backtest Execution Function

def run_strategy000_backtest():
    """
    Execute backtest for Strategy000 using S&P 500 historical data
    """
    print("=== INITIALIZING STRATEGY000 BACKTEST ===")
    
    # Initialize Cerebro
    cerebro = bt.Cerebro()
    
    # Load the preprocessed data
    df = pd.read_csv('spy_data_strategy000.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Get date range from the DataFrame  
    fromdate = df['Date'].min()
    todate = df['Date'].max()
    
    print(f"📊 Using S&P 500 data from {fromdate.strftime('%Y-%m-%d')} to {todate.strftime('%Y-%m-%d')}")
    print(f"📈 Total trading days: {len(df):,}")

    # Load the data into Backtrader
    data = btfeeds.GenericCSVData(
        dataname='spy_data_strategy000.csv',
        dtformat=('%Y-%m-%d'),
        datetime=0,  # Date column
        open=1,      # Open column
        high=2,      # High column  
        low=3,       # Low column
        close=4,     # Close column
        volume=5,    # Volume column
        openinterest=-1,  # No open interest data
        headers=True,     # Skip header row
        fromdate=fromdate,
        todate=todate,
    )

    # Add data and strategy to Cerebro
    cerebro.adddata(data)
    cerebro.addstrategy(Strategy000)

    # Set initial capital and commission
    initial_capital = 100000.0
    cerebro.broker.setcash(initial_capital)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

    # Add performance analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annual_returns")

    print(f"💰 Starting Portfolio Value: ${cerebro.broker.getvalue():,.2f}")
    
    # Run the backtest
    print("\n🚀 RUNNING STRATEGY000 BACKTEST...")
    results = cerebro.run()
    
    # Print final results
    final_value = cerebro.broker.getvalue()
    total_return = ((final_value - initial_capital) / initial_capital) * 100
    
    print(f"\n=== STRATEGY000 BACKTEST RESULTS ===")
    print(f"💰 Final Portfolio Value: ${final_value:,.2f}")
    print(f"📈 Total Return: {total_return:.2f}%")
    print(f"💵 Absolute Profit: ${final_value - initial_capital:,.2f}")

    # Extract detailed analytics
    strat = results[0]
    
    # Trade Analysis
    print('\n=== TRADE ANALYSIS ===')
    trade_analysis = strat.analyzers.trades.get_analysis()
    if trade_analysis and hasattr(trade_analysis, 'total'):
        total_trades = getattr(trade_analysis.total, 'total', 0)
        print(f"📊 Total Trades: {total_trades}")
        
        # Winning trades
        if hasattr(trade_analysis, 'won'):
            won_trades = getattr(trade_analysis.won, 'total', 0)
            win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
            print(f"✅ Winning Trades: {won_trades} ({win_rate:.1f}%)")
        
        # Losing trades  
        if hasattr(trade_analysis, 'lost'):
            lost_trades = getattr(trade_analysis.lost, 'total', 0)
            print(f"❌ Losing Trades: {lost_trades}")
    else:
        print("📊 No trades executed")

    # Performance Metrics
    print('\n=== PERFORMANCE METRICS ===')
    
    # Sharpe Ratio
    sharpe_analysis = strat.analyzers.sharpe.get_analysis()
    if sharpe_analysis and 'sharperatio' in sharpe_analysis:
        sharpe_ratio = sharpe_analysis['sharperatio'] or 0
        print(f"📊 Sharpe Ratio: {sharpe_ratio:.4f}")

    # Drawdown Analysis
    drawdown_analysis = strat.analyzers.drawdown.get_analysis()
    if drawdown_analysis:
        max_dd = drawdown_analysis.get('max', {}).get('drawdown', 0)
        max_dd_money = drawdown_analysis.get('max', {}).get('moneydown', 0)
        print(f"📉 Max Drawdown: {max_dd:.2f}%")
        print(f"💸 Max Drawdown ($): ${max_dd_money:,.2f}")

    # Returns Analysis
    returns_analysis = strat.analyzers.returns.get_analysis()
    if returns_analysis:
        total_return_analyzer = returns_analysis.get('rtot', 0) * 100
        avg_return = returns_analysis.get('ravg', 0) * 100
        print(f"📈 Total Return (Analyzer): {total_return_analyzer:.2f}%")
        print(f"📊 Average Daily Return: {avg_return:.4f}%")

    return cerebro, results, {
        'final_value': final_value,
        'total_return': total_return,
        'initial_capital': initial_capital
    }

print("✓ Strategy000 backtest function ready!")
print("✅ Ready to execute Strategy000 backtest")

✓ Strategy000 backtest function ready!
✅ Ready to execute Strategy000 backtest


In [6]:
# Execute Strategy000 Backtest
print("🎯 EXECUTING STRATEGY000 BACKTEST...")
print("=" * 60)

try:
    cerebro, results, performance_summary = run_strategy000_backtest()
    
    print("\n" + "=" * 60)
    print("🎉 STRATEGY000 BACKTEST COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    
    print(f"\n📋 SUMMARY:")
    print(f"   💰 Initial Capital: ${performance_summary['initial_capital']:,.2f}")
    print(f"   💰 Final Value: ${performance_summary['final_value']:,.2f}")  
    print(f"   📈 Total Return: {performance_summary['total_return']:.2f}%")
    
    if performance_summary['total_return'] > 0:
        print(f"   ✅ Strategy is PROFITABLE!")
    else:
        print(f"   ❌ Strategy shows LOSS")
        
except Exception as e:
    print(f"❌ Error running backtest: {str(e)}")
    print("💡 Make sure the S&P 500 data file exists at the specified path")

🎯 EXECUTING STRATEGY000 BACKTEST...
=== INITIALIZING STRATEGY000 BACKTEST ===
📊 Using S&P 500 data from 1789-07-01 to 2025-09-17
📈 Total trading days: 39,530
💰 Starting Portfolio Value: $100,000.00

🚀 RUNNING STRATEGY000 BACKTEST...
❌ Error running backtest: not enough values to unpack (expected 3, got 0)
💡 Make sure the S&P 500 data file exists at the specified path
❌ Error running backtest: not enough values to unpack (expected 3, got 0)
💡 Make sure the S&P 500 data file exists at the specified path


## Framework for Additional Strategies

The notebook now contains a complete implementation of **Strategy000** translated from MQL4 to Python Backtrader.

### Next Steps for Additional Strategies:

1. **Strategy001**: RVI + Envelopes
2. **Strategy002**: Stochastic + RSI  
3. **Strategy003**: Entry Time + Bears Power
4. **Strategy004**: MACD + Stochastic
5. **Strategy005**: Williams %R + ADX
6. **And more...**

### Parser Template for Future Strategies:

Each additional strategy will follow this pattern:
1. **Parse MQL4 code** → Extract indicator parameters
2. **Create Backtrader class** → Implement with talib indicators  
3. **Add entry/exit logic** → Translate boolean combinations
4. **Execute backtest** → Use same S&P 500 data
5. **Compare results** → Analyze performance metrics

The framework is now ready for expanding to all 17 strategies found in the MQL4 code.